### Importing Libraries

In [2]:
import base64
import json
import os
import shutil
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import tensorflow as tf
import tensorflow_hub as hub

2023-05-19 14:47:01.615248: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-19 14:47:01.661168: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-19 14:47:01.663208: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 14:47:02.691992: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Building Model

In [3]:
CLASSES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3

BATCH_SIZE = 32

FILE_DIR = 'datasets/flowers'

In [ ]:
TRAIN_PATTERN = FILE_DIR + "/train*"
EVAL_PATTERN = FILE_DIR + "/eval*"


def parse_example(example):
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    example["image"] = tf.io.decode_jpeg(example["image"], channels=3)
    example["image"] = tf.image.resize(
        example["image"], [IMG_HEIGHT, IMG_WIDTH]
    )
    example["image"] = example["image"] / 255
    return example["image"], example["label"]


train_ds = (
    tf.data.TFRecordDataset(tf.io.gfile.glob(TRAIN_PATTERN))
    .map(parse_example)
    .batch(BATCH_SIZE)
)
eval_ds = (
    tf.data.TFRecordDataset(tf.io.gfile.glob(EVAL_PATTERN))
    .map(parse_example)
    .batch(10)
)

In [ ]:
module_selection = "mobilenet_v2_100_224"
module_handle = "https://tfhub.dev/google/imagenet/{}/feature_vector/4".format(
    module_selection
)

transfer_model = tf.keras.Sequential(
    [
        hub.KerasLayer(module_handle, trainable=True),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(
            len(CLASSES),
            activation="softmax",
            kernel_regularizer=tf.keras.regularizers.l2(0.0001),
        ),
    ]
)

transfer_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
transfer_model.fit(
    train_ds,
    epochs=5,
    validation_data=eval_ds,
)

### Exporting model

In [ ]:
shutil.rmtree("export", ignore_errors=True)
os.mkdir("export")
transfer_model.save("export/flowers_model")

In [ ]:
!saved_model_cli show --dir export/flowers_model --tag_set serve --signature_def serving_default

### Prediction

In [ ]:
filenames = [
    "gs://asl-public/data/flowers/jpegs/10172567486_2748826a8b.jpg",
    "gs://asl-public/data/flowers/jpegs/10386503264_e05387e1f7_m.jpg",
    "gs://asl-public/data/flowers/jpegs/10391248763_1d16681106_n.jpg",
    "gs://asl-public/data/flowers/jpegs/10712722853_5632165b04.jpg",
    "gs://asl-public/data/flowers/jpegs/10778387133_9141024b10.jpg",
    "gs://asl-public/data/flowers/jpegs/112334842_3ecf7585dd.jpg",
]


def preprocess(img_bytes):
    img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    return img


def read_from_jpegfile(filename):
    img = tf.io.read_file(filename)
    img = preprocess(img)
    return img


serving_model = tf.keras.models.load_model("export/flowers_model")
input_images = [read_from_jpegfile(f) for f in filenames]

f, ax = plt.subplots(1, 6, figsize=(15, 15))
for idx, img in enumerate(input_images):
    ax[idx].imshow(img.numpy())
    batch_image = tf.expand_dims(img, axis=0)
    batch_pred = serving_model.predict(batch_image)
    pred = batch_pred[0]
    pred_label_index = tf.math.argmax(pred).numpy()
    pred_label = CLASSES[pred_label_index]
    prob = pred[pred_label_index]
    ax[idx].set_title(f"{pred_label} ({prob:.2f})")

### Additional Signature

In [ ]:
def postprocess(pred):
    top_prob = tf.math.reduce_max(pred, axis=[1])
    pred_label_index = tf.math.argmax(pred, axis=1)
    pred_label = tf.gather(tf.convert_to_tensor(CLASSES), pred_label_index)

    return {
        "probability": top_prob,
        "flower_type_int": pred_label_index,
        "flower_type_str": pred_label,
    }


@tf.function(input_signature=[tf.TensorSpec([None,], dtype=tf.string)])
def predict_from_filename(filenames):

    input_images = tf.map_fn(
        read_from_jpegfile, filenames, fn_output_signature=tf.float32
    )

    batch_pred = transfer_model(input_images)  # same as model.predict()

    processed = postprocess(batch_pred)
    return processed



In [ ]:
@tf.function(input_signature=[tf.TensorSpec([None,], dtype=tf.string)])
def predict_from_b64(img_bytes):

    input_images = tf.map_fn(
        preprocess, img_bytes, fn_output_signature=tf.float32
    )

    batch_pred = transfer_model(input_images)  # same as model.predict()

    processed = postprocess(batch_pred)
    return processed

In [ ]:
transfer_model.save(
    "export/flowers_model_with_signature",
    signatures={
        "serving_default": predict_from_filename,
        "predict_base64": predict_from_b64,
    },
)

In [ ]:
!saved_model_cli show --dir export/flowers_model_with_signature --tag_set serve --signature_def serving_default

In [ ]:
serving_fn = tf.keras.models.load_model(
    "export/flowers_model_with_signature"
).signatures["serving_default"]

pred = serving_fn(tf.convert_to_tensor(filenames))

for k in pred.keys():
    print(f"{k:15}: {pred[k].numpy()}")


In [ ]:
!saved_model_cli show --dir export/flowers_model_with_signature --tag_set serve --signature_def predict_base64